# Outline of the session. What will we learn?

This is an introductory session, where we learn the basics of the tools that we will use in the following sessions.

- Astropy Main ingredients: units and constants
- Astropy coordinates
- Astroquery: Vizier queries
- Simple Gaia queries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# These will be loaded during the tutorial, when needed:
# from astropy import units as u
# from astropy.coordinates import SkyCoord
# from astroquery.simbad import Simbad
# from astropy.coordinates import Angle
# from astroquery.vizier import Vizier
# from astroquery.gaia import Gaia

# Astropy Main ingredients
The Astropy units and quantities documentation can be found [here](https://docs.astropy.org/en/stable/units/index.html)

In [ ]:
from astropy import units as u

In [ ]:
d = 10 * u.meter  
d

In [ ]:
type(d)

A `Quantity` is formed by a `value` (can be a number, a list, a numpy array, etc) and the associated `unit`.

In [ ]:
print(d.value)
print(d.unit)

In [ ]:
a = [30,60,90]*u.deg
a

In [ ]:
t = np.arange(6)*u.h
t

### Operations with Quantities

You can operate with `Quantities` of the same type, even if the units are different, and astropy will make sure that the operations are consistent. When adding quantities, the result will be expressed in the units of the first quantity.

In [ ]:
d + 15*u.cm

In [ ]:
a + 15*u.arcmin

In [ ]:
t + 525*u.second

In [ ]:
#d+a

### Unit conversion and operations

The `Quantity` objects have a method to convert to any units available in astropy: `.to()`, which accepts the name of the unit as a string. 

In [ ]:
d.to('km')

In [ ]:
d.to(u.km)

In [ ]:
a.to('milliarcsecond')

astropy understands a large variety of units. You can find all units available and their name aliases in [module-astropy.units](https://docs.astropy.org/en/stable/units/index.html#module-astropy.units).

In [ ]:
100*u.km/(t + 1*u.min)

In [ ]:
v = 100 * u.meter / u.second
v

In [ ]:
v.to(u.km/u.day)

In [ ]:
v.to(u.pc/u.yr)

### Other operations
We can also apply more complex functions, for example any of the `numpy` statistical functions will work.


In [ ]:
print(t)
np.mean(t)

In [ ]:
angle_deg = 33*u.deg
angle_arcsec = angle_deg.to(u.arcsec)
print(np.cos(angle_deg), angle_deg)
print(np.cos(angle_arcsec), angle_arcsec)

In [ ]:
r = np.random.randn(4,4)*u.cm
print(r)
np.min(r).to('m')

In [ ]:
Emin = 100*u.keV
Emin

In [ ]:
Emax = 10*u.MeV
Emax

In [ ]:
Emax/Emin

In [ ]:
np.log(Emax/Emin)

An alternative way to create a `Quantity` object, which is much faster for large datasets

In [ ]:
x = u.Quantity(np.arange(10), unit='m')
x

### ⛏ Exercise 1.1

Consider the triangle rectangle with angles 90 deg, 40 deg and 50 deg. Create 3 variables `a1`, `a2`, `a3` and assign the three angles in degrees.

Convert `a1` to arcmin, `a2` to arcsec and `a3` to milliarcsec and assign them to variables `a1_arcmin`, `a2_arcsec` and `a3_mas`, respectively, and print them.  

Compute the sum of the three converted angles, `a_sum`. What are the units of this variable?

Verify that the sum is 180 deg by using `a_sum == 180*u.deg`

### Physical constants
More details on the available constants can be found [here](https://docs.astropy.org/en/stable/constants/index.html#module-astropy.constants).

In [ ]:
from astropy import constants as const
# speed of light
speed_of_light = const.c
speed_of_light

In [ ]:
print(speed_of_light)

In [ ]:
const.M_sun

In [ ]:
const.M_sun.to('g')

In [ ]:
# distance sun - earth
distance_sun = 1 * u.au

# time
time_sun = distance_sun / speed_of_light
time_sun

In [ ]:
time_sun.to('min')

In [ ]:
print(f"SI: {distance_sun.si}")
print(f"CGS: {distance_sun.cgs}")

### ⛏ Exercise 1.2
Assuming that the distance to the Galactic Center is 8 kpc, compute the time it takes for light to travel from the Galactic Center to the Earth, in years.

# Astropy Angles and Coordinates

In [ ]:
from astropy.coordinates import Angle

Angle(24*u.deg)

In [ ]:
a = Angle(1*u.deg)
print(a.deg)
print(a.arcmin)
print(a.arcsec)

In [ ]:
angles = Angle([1, 4, 6, 15, -3, 2, 10, 2, 0]*u.deg)
angles

In [ ]:
angles.to('arcmin')

In [ ]:
angles.sort()  # The variable angles will be modified insitu.
angles.arcsec

In [ ]:
angles.sum()

Let's work with coordinates. More details on the use of coordinates can be find [here](https://docs.astropy.org/en/stable/coordinates/index.html).

In [ ]:
from astropy.coordinates import SkyCoord

Note that it is important to define the reference frame (ICRS, FK5, Galactic, etc) to correctly identify astronomical coordinates. More details in [astropy Using and Designing Coordinate Frames](https://docs.astropy.org/en/stable/coordinates/frames.html). If not specified, the default is `ICRS`. The frame classes that are built in to astropy are ICRS, FK5, FK4, FK4NoETerms, and Galactic.

In [ ]:
c = SkyCoord(ra=10.625*u.degree,
             dec=41.2*u.degree,
             frame='icrs')
c

Multiple ways to initialize the coordinates, and astropy is usually very clever interpreting what you want.

In [ ]:
c1 = SkyCoord(10.625, 41.2, frame='icrs', unit='deg')
c2 = SkyCoord('00h42m30s', '+41d12m00s', frame='icrs')
c3 = SkyCoord('00h42.50m', '+41d12m')
c4 = SkyCoord('00 42 30 +41 12 00', unit=(u.hourangle, u.deg))
c5 = SkyCoord('00:42.5 +41:12', unit=(u.hourangle, u.deg))
print(c1, c2, c3, c4, c5)

The coordinates can be accessed by R.A. or Declination, and can be converted to different units.

In [ ]:
print(c.ra)
print(c.dec.arcsec)
print(c.ra.radian)
print(c.ra.hms)
print(c.dec.dms)

You can also initialize arrays of coordinates. Just parse a `numpy` array or a python `list` to the variables.

In [ ]:
cs = SkyCoord(np.random.uniform(0, 180, 20)*u.deg,
              np.random.uniform(-90, 90, 20)*u.deg,
              frame='galactic')
cs

In [ ]:
cs.to_string('hmsdms')

In [ ]:
plt.plot(cs.l, cs.b, 'o')

Including observation time and convert to observational `QTable`. `QTable` and `Table` are the basic astropy table representations. We will explain them in detail in tutorial 02.

In [ ]:
from astropy.time import Time

In [ ]:
sc = SkyCoord(ra=[40, 70]*u.deg,
              dec=[0, -20]*u.deg,
              obstime=Time([2000.2, 2010.123], format='decimalyear'))
tab =  sc.to_table()
tab

In [ ]:
print(tab['obstime'])
print(tab['obstime'].mjd)
print(tab['obstime'].iso)
print(tab['obstime'].fits)

Coordinates: transformation of the reference system

In [ ]:
c_icrs = SkyCoord(ra=10.68458*u.degree,
                  dec=41.26917*u.degree,
                  frame='icrs')
c_galactic = c_icrs.galactic  
c_galactic

In [ ]:
c_galactic.transform_to('icrs')

Distance between two coordinates

In [ ]:
c1 = SkyCoord(ra=10*u.degree, dec=9*u.degree, frame='icrs')
c2 = SkyCoord(ra=11*u.degree, dec=10*u.degree, frame='fk5')
c1.separation(c2)  # Differing frames handled correctly  

Search coordinates by source name

In [ ]:
SkyCoord.from_name("PSR J1012+5307")

### ⛏ Exercise 1.3

- Create a `SkyCoord` instance names `pos1` that points to R.A., Dec. '00h10m20s', '+10d20m30s' in the ICRS frame.
- Create a another `SkyCoord` instance `pos2` with 3 different positions: R.A., Dec expressed in arcmin in the `fk5` frame as:
  - [100, -20],
  - [150, 0]
  - [200, 20]

- Compute the angular separation between `pos1` and each of the coordinates in `pos2`. Express them in arcmin.
- What is the minimum, maximum and mean separation in arcsec, arcmin and deg, respectively, between `pos1` and the objects in `pos2`?


### ⛏ Exercise 1.4
Find the separation in arcmin between M51 and the point of the Sky in Galactic Coordinates l=10h25m47.0s and b=+10deg in galactic reference frame.

### 🌪 Additional fun

- Make a python list with the names of the 110 Messier objects.
- Create a table with their coordinates, as provided by `SkyCoord`.
- Compute the angular separation between M1 and the rest of the objects.
- Find the coordinates of the closest and the furthest Messier object from M1?
- What is the average position of all Messier object coordinates?
- What is the closest Simbad object to that average position? Use `Simbad.query_region` [query region](https://astroquery.readthedocs.io/en/latest/simbad/simbad.html#query-a-region).
- Make an all-sky plot where you show the positions of the Messier objects, adding an annotation to name each one.

### 🌪 Additional fun

- Generate two arrays of 100 values following a pattern, for example a linear, a trigonometric function or a combination of both.
- Add Gaussian noise to those arrays using `np.random.normal`. Use a `sigma` that is conmensurable to the magnitude of the values.
- Generate an array of 100 MJD values.
- Create a `SkyCoord` object combining the first two arrays as coordinates and the third as `obstime`.
- Convert the coordinates to a `QTable` and show the first few rows.
- Plot the projections of R.A., Dec. and obstime


# Astroquery
[Astroquery](https://astroquery.readthedocs.io/en/latest/) is a coordinated package of astropy.

It provides easy access to many different [services](https://astroquery.readthedocs.io/en/latest/#available-services), [catalogs](https://astroquery.readthedocs.io/en/latest/#catalogs) and [archives](https://astroquery.readthedocs.io/en/latest/#archives) that follow Virtual Observatory standards. In particular it provides access to Simbad, Vizier, NED, Gaia, [ESASky](https://sky.esa.int/) and many others.

Some [example](https://ioa-coding.github.io/codecorners/2017_10_24_CC02_astroquery.html)

## Vizier Table Discovery

In [ ]:
from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1

In [ ]:
catalog_list = Vizier.find_catalogs('fermi 4fgl')
['{}: {}'.format(k, v.description) for k, v in catalog_list.items()]

In [ ]:
Vizier.get_catalogs('J/ApJS/247/33')

In [ ]:
cat = Vizier.get_catalogs('J/ApJS/247/33')[0]

In [ ]:
cat

Good! Note that the tables contain unit information, so there will be no ambiguity when using the data from Vizier tables. But we can get more information, in particular the description of each column.

In [ ]:
cat.info

In [ ]:
ra_deg = Angle(cat['RAJ2000'])
de_deg = Angle(cat['DEJ2000'])
ra_deg = ra_deg.wrap_at('180d')

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(ra_deg.rad, de_deg.rad, s=1)
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)

### ⛏ Exercise 1.5
- Search Vizier catalogs related to the keyword `carmenes` (or any other you are interested).
- Select the ID of the catalog named "324 CARMENES M dwarfs velocities" and download the first table.
- Use `info` to find out the name of the columns with the coordinates.
- Find out the minimum, maximum and mean period in the sources in this catalog.
- Make an all-sky plot of the sources in this catalog.

## Query a Simbad object

In [ ]:
result = Vizier.query_object("NGC 6670")
print(result)

## Quick access to data from the Gaia catalog

The library we will use to get Gaia data is [Astroquery](https://astroquery.readthedocs.io/en/latest/). Astroquery provides `Gaia`, which is an [object that represents a connection to the Gaia database](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html).

We can connect to the Gaia database like this:

In [ ]:
from astroquery.gaia import Gaia
Gaia.ROW_LIMIT = 50
Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source" # Select early Data Release 3
# Gaia.MAIN_GAIA_TABLE = "gaiadr2.gaia_source"  # Reselect Data Release 2, this is the default 

We can easily query the Gaia catalog using the `astroquery` function `Gaia.query_object_async`, which can send query jobs to the Gaia TAP+ service. Note that the filtering options in this function are very limited, so a query may have too many results to handle. Verify your query with a low number of the parameter `Gaia.ROW_LIMIT` first. For full query functionality we should used the [Astronomical Data Query Language (ADQL)](https://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html), which we will use in Session 3. For the moment the simple query is enough for our purposes.

To do a basic query on the *Gaia* catalogue we just need a pointing centre and the (width, height), or the radius, of the region of interest. This is a clear case where you could build a python function to retrieve data based only on those four parameters.

In [ ]:
sgra_coord = SkyCoord.from_name("Sgr A*")
search_radius = 50*u.arcsec

sgra_coord.to_string('hmsdms')

In [ ]:
# Run the actual query to the catalogue
gaia_sgra = Gaia.query_object_async(coordinate=sgra_coord,
                                    radius=search_radius, 
                                    columns=['ra','dec'])

In [ ]:
ra_relative  = gaia_sgra['ra'].to(u.arcsec) - sgra_coord.ra
dec_relative = gaia_sgra['dec'].to(u.arcsec) - sgra_coord.dec


plt.plot(ra_relative, dec_relative, 'ok')
plt.plot(0, 0, '+', ms=20)

In [ ]:
my_cols = data0.columns

### ✨ Exercise 1.6

- Define the central coordinates of the cluster, `cluster_coord` of interest R.A. 130.025 deg and Dec. 19.98333 deg.
- Define the variable `width_deg` and `height_deg` to be 3 and 2 degrees, respectively.
- Create a table `data0` using `Gaia.query_object_async` and setting the variables `coordinate`, `width`, `height` and `columns`. The variable columns is defined with:
```
gaia_columns = ['ra','dec','ra_error','dec_error','parallax','parallax_error','parallax_over_error','pmra','pmra_error','pmdec','pmdec_error','matched_observations','phot_g_mean_flux','phot_g_mean_flux_error','phot_g_mean_mag','phot_bp_mean_flux','phot_bp_mean_flux_error','phot_bp_mean_mag','phot_rp_mean_flux','phot_rp_mean_flux_error','phot_rp_mean_mag','bp_rp','bp_g','g_rp','radial_velocity','radial_velocity_error','teff_val','radius_val']
```
- How many entries does the table have? And how many columns?

In [ ]:
Gaia.ROW_LIMIT = -1  # To get all results
Gaia.MAIN_GAIA_TABLE = "gaiaedr3.gaia_source"    # Select early Data Release 3

In [ ]:
# These are the columns we are interested in:
gaia_columns = ['ra','dec','parallax','parallax_error','pmra','pmra_error','pmdec','pmdec_error','astrometric_matched_transits','phot_g_mean_flux','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','bp_rp','bp_g','g_rp','dr2_radial_velocity','dr2_radial_velocity_error','dr2_rv_template_teff']

### 🌪 Additional fun

Instead of using the `Gaia` module, search Vizier with `Vizier.find_catalogs('edr3')` and retrieve the main Gaia EDR3 table with `astroquery.Vizier`. **Important** make sure to use `column_filters` to select the region you want. More info in https://astroquery.readthedocs.io/en/latest/vizier/vizier.html#query-a-region

# Conclusion. In future episodes...

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(14,8))

ax.scatter(data0['ra'], data0['dec'], s=data0['phot_g_mean_flux']/1e5);
ax.set_aspect('equal')

# Here we invert the direction of the right ascension axis
ax.invert_xaxis()

ax.set_xlabel('Right Ascension [deg]')
ax.set_ylabel('Declination [deg]');
